In [24]:
import pandas as pd
import re
import json
import glob
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define range for file, friends change this help me download when you're free lel
start_int = 0
end_int = 100

In [25]:
# Combine CSV, generate output

files = glob.glob("Clean_data/*.csv")

csv = []

for file in files:
    df = pd.read_csv(file, index_col=None, header=0)
    csv.append(df)

df = pd.concat(csv, axis=0, ignore_index=True)

In [26]:
print(df.shape, "\n",df.columns)

(195933, 27) 
 Index(['Unnamed: 0', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'current_currency',
       'deadline', 'fx_rate', 'goal', 'id', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'source_url', 'spotlight', 'staff_pick',
       'state', 'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')


In [28]:
urls = df.urls
ids = df.id

urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
urls = list(map(lambda url:url.replace("?ref=discovery_category_newest","/comments"),urls_))
#urls = urls_

#get url
#url = df[df.state == "successful"].urls.reset_index().urls[1]
#url = json.loads(url)
#url = url["web"]["project"]
#url

In [29]:
#path = "Webdriver/chromedriver.exe"
#driver = webdriver.Chrome(path)

from selenium.webdriver.support import ui
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome('/Users/amandatan/Downloads/chromedriver') 


comments = []

for i in tqdm(range(start_int,end_int)):
    driver.get(urls[i])
#driver.get(url)

#wait = ui.WebDriverWait(driver, 10)
#wait.until(lambda driver: driver.find_element_by_xpath('//a[@id="comments-emoji"]')).click()
#driver.find_element_by_id('comments-emoji').click()

    comments_raw = WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.XPATH,"//div[@id='react-project-comments']"))
    )
    #for comment in comments_raw:

    while True:
        if len(comments_raw.text) >0:
            comments.append(comments_raw.text)
            break


driver.quit()

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/amandatan/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache
100%|██████████| 10/10 [00:25<00:00,  2.58s/it]


In [30]:
comments

['Only backers can post comments. Log in\nNo comments yet.',
 'Only backers can post comments. Log in\nNo comments yet.',
 'Only backers can post comments. Log in\nGilles Jankowski\nalmost 4 years ago\nGilles Jankowski\n26 op Reilend\nL-7391 Blaschette',
 'Only backers can post comments. Log in\nNo comments yet.',
 'Only backers can post comments. Log in\nYannig Marchegay\nover 9 years ago\nCome on, only 9 backers? :-(',
 'Only backers can post comments. Log in\nNo comments yet.',
 'Only backers can post comments. Log in\nNo comments yet.',
 'Only backers can post comments. Log in\nLisa TeslerSuperbacker\nover 3 years ago\nTried to see TTWS at Kaaboo last weekend. The stage area was too small, and I couldn\'t get anywhere even close! I literally couldn\'t see a thing. I had a Hang Five pass, but it was so jammed I couldn\'t even get to that special entrance!\nBummed to have missed them, but glad to see how popular their set was. I feel like I had something to do with that, as one of th

In [ ]:
df_export = pd.DataFrame({"index":list(range(start_int,end_int)),
                         "comments":comments}, index = None)
filename = "Comments_" + str(start_int) + "-" + str(end_int)
df_export.to_csv("Output/Comments/{}.csv".format(filename))